<a href="https://colab.research.google.com/github/garymm/misc-data/blob/master/pytorch_onnx_changes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install PyDriller gitpython
import os
import pandas
import pathlib
from pydriller import Repository
import git

In [ ]:
# Recent commits that touched paths containing "onnx".
# Filtering this ahead of time speeds things up a lot.
REVISIONS = [
  "4b9135523229c9d78ba20c696b86670bd765a16d", "f1ce7f4b7f7e668edca1bf12099ce5cdb5d31d69", "3979cb0656fe2d8b0445768a769bd624b10778b5", "dfe85d6fd75dd808f89afefd5abec58b97e684e5", "3453aa44c1ea5239f9cb8b4811976339da47ed83", "eae84f0d5de1b1a9cab8d18a28d231fdc5e84b96", "599f5058cf0726d163e04e2d542530f63fcf439c", "cab4849463f962b5498b795e00886a4ed0a4dd00", "ef32a29c975c0665bce1d6a6f8d4e45d2e4d57c5", "2c17b6a0fe254f18f3c969cb0e68920341ec600b", "1aabb8f98c42ff262473ff1362e166b3fb2c9f25", "0a6828a3063240ba5ec426e236d195a372fa51b4", "b27fc0ff855fd19759be5b707dc2b4387eb40599", "57c9355a0df3039f884ec12d721142ad8cde0981", "b8c96e6b0808eb71a4bf87d1d5e075ccf8d485e9", "d101816fdd11ce62972e2323ddcdbbafd0b94692", "51d14b6859babbc7646cff1d58852bc4e8efc18f", "0d5527de7a7d1f6d4802d7f7c0e2e3ba8588b784", "09a8f22bf917727bc9179443185fdda875e04147", "a7f4f80903b331ecb9485ccf534df1dfc5748cff", "4ca4640bae603d2b765a4063f6475d98f0dc218b", "9db64e6e56e0b0ead8c120ebeebdd112e673a649", "3113a1de4ac75bb397911ab1ae0bd3e98de89e03", "3d12ab452e18ce6146a7ddd77a7ac967f7cda5b8", "c034bce979676e64d3cbd14f9db81690cf489100", "0d11dbf5119e1e4c69016ed1527bcf5013b208af", "520f90f35964636be5da0fa7b56cb1d1f2aa9353", "51cd89ecc6d6691fff07319c8666633a5cfd5a44", "01374d69e46ec722a46007b1e2f75b7f33e04329", "8e29863a0d01499018556362e823962e78b73c9d", "bfe7728f186575909111eeb098524b9eca20af5d", "346dc88bfa271403aab20affd5f8806c4370a06b", "2b0f481d3f95dc7356df8b0ef08ec840fac9ed01", "9e56314d2cecf079a487fb9f6799183736686d48", "dc0071dfa599db6d73b5abf366148b93d988e2dc", "ac9e79e561e8691a8301f6529809b8267cc6ee20", "3bc8a2264d23cbe1e3c7bfa59a33ca26a82fabf8", "dd3bd0286bb6c2af833fac24c8756ad84b35d7d5", "fee7e8b91d4434b976a339330bfa89bd827ab9ec", "1de3525ca8e141309791d34d5456ece771b3d045", "5e83c62a9ec2ee80af06401b1ddeb70edde277bc", "35521a26299b288d4c077540e93d9bc0649fe2e5", "3a66a1cb99d7e1c1fe7abf8f390e177b9183a436", "20430932177725efdc9bff53500e2f2397e4d20b", "33eea146eec170d6f8cecbef5d99c38cc3237a1e", "b87d3fa4326a460b161b01f276dd97252b7f5f48", "eac02f85cf21787879212275c93cbe206372a97d", "4cb534f92ef6f5b2ec99109b0329f93a859ae831", "a90a3acbeec7deebecf7baf53c8f0fdf7b37e5ae", "51bca2ca4df42829acfb99ad57419636cf0449be", "3fbc15410ad4723d8ee23ac79275a03677a071da", "88bd0510ef9e691edd22294e8b8166003ec2caa9", "913f1f75b33be186474c65cafd5daad91a6e5855", "36828aa0fff1bf57679ac62693b683ecb796e1c4", "24ff92f76d9998455258257334fca490b558cbd8", "818ce1d0d2fc8cd0ff5ed811294f529da598f0d1", "75024e228ca441290b6a1c2e564300ad507d7af6", "88fbbb4165da19ce83c7b22d391744c4cdf30f4a", "1e449694a3c2b5ecd6a0045fc8a0c0f6315ee40e", "0b0fca3c59b090efbddc6afd590efd904c027ed1", "90e63cc41f06f6f5920916492921704e660a853d", "a31fd7f45373be3f01e5b822be513a164927c64b", "5a455dc717f8cd935117c25e6678f0677b83ed93", "f804b65d4e822aedfe7814b42dc9b2e1bc6d18d7", "9986b109d2166cab7d2b0f83572dc196825c29d0", "75995e4bf621a8ae079ba0faa12e25a4ad4fa002", "e3900d2ba5c9f91a24a9ce34520794c8366d5c54", "6ec71ed4f97d6b3f0ceb5b39cd924b76ecf40ac8", "3fe4718d164b4231684ff995ba13ca0d91d59c6d", "4753100a3baa96273204c361c8452afb7b59836f", "93bf0ae6fcc8546165d19ccc92c1ae6fed407de9", "e6bfff679d38e63f8b786061155f0850f63337ba", "add49e7e4e6e0b24665ec5f086258394706f1a37", "d6909732954ad182d13fa8ab9959502a386e9d3a", "2ee02b30b11b765839aa4020def1f84d738f8e42", "980d6f258912e8d4474c5ccfe1a1e4f88f226af5", "c0ac0fef4e95ef9e441b1b876e157d31254262dc", "041b4431b28eb24335189979e9a110dc9892dfe4", "7824d8277aebc20ac8c21c5f4b52a29182c24816", "40869884cd33e7172552a583402508d02ff0d509", "c5f3d92816789a8336cb955d54e60467b39a7d0d", "cb0cee4a3dfe71877337fec688fdfc96f0293730", "849dcb8b69eec9736a7e4c64eacd4307cd406421", "cd9dd653e98534b5d3a9f2576df2feda40916f1d", "ce48b1406095f8cc76703366bbba40d7ac828343", "5bcbbf537327f6e8328289c25a3a453a2444d984", "59d1f08b4ce0485b6aed6a9a6c165354ff019769", "645119eaefd0dcf1afc54e9ad58678b5245dea78", "3519625a3482418de0163eb4caa870af4fd6bad4", "27048c1dfa80effabf17b8dca66cd2724dd502f8", "ad8d1b2aaaf2ba28c51b1cb38f86311749eff755", "f6df18f6ca444561b4108d746ed7b579a78c0294", "5b648ef909fbf89c53f28ebc1b3bd2f4fde168c5", "a51f130d377e680e242908a5b9d888b407f31b3a", "ee4ce8e9d9a439b9a87fcfa6abdcb0f7f3f62b11", "a572f70f2f4d07a5ded047c4a9f9d1db6feb7e47", "705131c5d307b33b7251e389b3414c7733610ae9", "a6a811f23aa0eed68022b06591c6021215d0dee4", "76147b897cd1ec1486430a6bc7a9ef8521c2da86", "4c1d9e58c24fece936cf87aae6c1a57eda1cddaf", "8dab886d3b4e12258e60f4180b9714fdd4474913", "be344e9d880a17d7ffae7787cd0cfdaa90a19326", "7f17058894b3a85c7f14d943ca4976ca266cbd88", "57d1df071f6c7199a01cdb3e15ea2be03ef96706", "38414d29a1b66e3b9a0b406160e042b3a898b964", "b9e900ee525eeaa601aefdeadaf89c6e1a3085e8", "3f9c803fe82e5bb5ccf30b82f5c129ba8c695208", "21f9a6da7db470597538319759ffb50d7206b3a5", "8a6df06a0ee1b5ac5c12279577af86d05ef04b5a",
]

In [ ]:
repo_dir = pathlib.Path("pytorch")
if not os.path.exists(repo_dir / ".git"):
  git.Repo.clone_from("https://github.com/pytorch/pytorch.git", repo_dir)
repo = Repository(str(repo_dir), only_commits=REVISIONS)

In [ ]:
def format_path(s):
  return f'<a href="https://github.com/pytorch/pytorch/blob/master/{s}">{s}</a>'


def format_commits(commits):
  links = sorted(
      [f'<a href="https://github.com/pytorch/pytorch/commit/{c}">{c[:6]}</a>'
      for c in commits])
  return str(",".join(links))


def commits_df(repo: Repository) -> pandas.DataFrame:
  paths = []
  commits = []
  is_onnx = []
  for commit in repo.traverse_commits():
    for mod in commit.modified_files:
      # skip deletions
      if not mod.new_path:
        continue
      paths.append(mod.new_path)
      commits.append(commit.hash)
      is_onnx.append("onnx" in mod.new_path)
  return pandas.DataFrame({
      "path": paths,
      "commit": commits,
      "is_onnx": is_onnx,
  })

def analyze_commits_df(
    df: pandas.DataFrame, except_commits=()) -> pandas.io.formats.style.Styler:
  if except_commits:
    df = df[~df["commit"].isin(except_commits)]
  non_onnx_counts = df[df["is_onnx"] == False][["path", "commit"]].groupby(
    "path", as_index=False).agg({"commit": [len, list]})
  non_onnx_counts.columns = ["path", "num_commits", "commits"]
  non_onnx_counts = non_onnx_counts.sort_values(
    by=["num_commits", "path"], ascending=False).reset_index(drop=True)

  return non_onnx_counts.style.format(
      {"path": format_path,
      "commits": format_commits}
  )

# All commits #

In [ ]:
df = commits_df(repo)
analyze_commits_df(df)

,path,num_commits,commits
0,aten/src/ATen/native/native_functions.yaml,12,"09a8f2,204309,2ee02b,33eea1,3fe471,4ca464,645119,980d6f,9db64e,a7f4f8,ef32a2,fee7e8"
1,torch/csrc/autograd/FunctionsManual.cpp,11,"041b44,09a8f2,204309,2ee02b,33eea1,3979cb,3a66a1,4cb534,980d6f,c0ac0f,d69097"
2,tools/autograd/derivatives.yaml,10,"09a8f2,204309,2ee02b,33eea1,3fe471,980d6f,9db64e,a7f4f8,ef32a2,fee7e8"
3,torch/testing/_internal/common_methods_invocations.py,8,"204309,2ee02b,3113a1,33eea1,35521a,5e83c6,75024e,980d6f"
4,torch/csrc/utils/python_arg_parser.cpp,8,"041b44,3979cb,4cb534,6ec71e,a7f4f8,c0ac0f,eac02f,ef32a2"
5,torch/_C/__init__.pyi.in,7,"0d5527,1de352,3f9c80,57d1df,818ce1,bfe772,c034bc"
6,test/test_fx.py,7,"09a8f2,3113a1,35521a,5e83c6,75024e,93bf0a,add49e"
7,torch/nn/functional.py,6,"09a8f2,3fe471,645119,a7f4f8,e3900d,ef32a2"
8,torch/csrc/jit/tensorexpr/kernel.cpp,6,"041b44,3979cb,3a66a1,4cb534,c0ac0f,eac02f"
9,torch/csrc/jit/runtime/static/ops.cpp,6,"041b44,3979cb,3fe471,4cb534,c0ac0f,d69097"


# Filtering out easy-to-split commits #

In [ ]:
# Adding or modifying ops and the ONNX support in the same commit
# in a way that could easily be split into two commits: first
# change pytorch, then change ONNX support.
# However this isn't totally non-problematic, as moving ONNX to
# a separate repo may lead to folks forgetting about it.
MODIFIED_OPS = {
    "09a8f22bf917727bc9179443185fdda875e04147",
    "3fe4718d164b4231684ff995ba13ca0d91d59c6d",
    "980d6f258912e8d4474c5ccfe1a1e4f88f226af5",
    "20430932177725efdc9bff53500e2f2397e4d20b",
    "645119eaefd0dcf1afc54e9ad58678b5245dea78",
    "33eea146eec170d6f8cecbef5d99c38cc3237a1e",
}
# I'm only including commits here that seem like they didn't need to be done
# atomically. Basically they were applying some transformation on every file in
# the repo, and missing out on files that happened to be in submodules wouldn't
# be a problem.
LARGE_SCALE_CLEANUPS = {
  "3a66a1cb99d7e1c1fe7abf8f390e177b9183a436",
  "4cb534f92ef6f5b2ec99109b0329f93a859ae831",
  "5bcbbf537327f6e8328289c25a3a453a2444d984",
  "93bf0ae6fcc8546165d19ccc92c1ae6fed407de9",
  "2ee02b30b11b765839aa4020def1f84d738f8e42",
  "6ec71ed4f97d6b3f0ceb5b39cd924b76ecf40ac8",
  "75024e228ca441290b6a1c2e564300ad507d7af6",
  "4753100a3baa96273204c361c8452afb7b59836f",
  "add49e7e4e6e0b24665ec5f086258394706f1a37",
  "e3900d2ba5c9f91a24a9ce34520794c8366d5c54",
  "eac02f85cf21787879212275c93cbe206372a97d",
  "f6df18f6ca444561b4108d746ed7b579a78c0294",
  "27048c1dfa80effabf17b8dca66cd2724dd502f8",
  "d6909732954ad182d13fa8ab9959502a386e9d3a",
}
# Commits that would have to modify ONNX first, then PyTorch,
# probably because they remove some API that is in use by
# ONNX code.
ONNX_FIRST = {
    "b87d3fa4326a460b161b01f276dd97252b7f5f48",
    "4ca4640bae603d2b765a4063f6475d98f0dc218b",
}
# These only touched ONNX files in the /caffe2 directory.
CAFFE2_ONLY = {
    "cab4849463f962b5498b795e00886a4ed0a4dd00",
}
analyze_commits_df(
    df, except_commits=LARGE_SCALE_CLEANUPS | MODIFIED_OPS | ONNX_FIRST | CAFFE2_ONLY)

,path,num_commits,commits
0,torch/_C/__init__.pyi.in,7,"0d5527,1de352,3f9c80,57d1df,818ce1,bfe772,c034bc"
1,torch/csrc/utils/python_arg_parser.cpp,5,"041b44,3979cb,a7f4f8,c0ac0f,ef32a2"
2,torch/csrc/jit/runtime/register_ops_utils.cpp,5,"041b44,3979cb,9db64e,c0ac0f,fee7e8"
3,torch/csrc/jit/passes/shape_analysis.cpp,5,"041b44,3979cb,9db64e,c0ac0f,fee7e8"
4,torch/csrc/jit/frontend/ir_emitter.cpp,5,"041b44,3979cb,9db64e,c0ac0f,fee7e8"
5,torch/csrc/jit/serialization/export.cpp,4,"041b44,3979cb,4b9135,c0ac0f"
6,torch/csrc/jit/passes/lower_tuples.cpp,4,"041b44,3979cb,b27fc0,c0ac0f"
7,tools/autograd/derivatives.yaml,4,"9db64e,a7f4f8,ef32a2,fee7e8"
8,aten/src/ATen/native/native_functions.yaml,4,"9db64e,a7f4f8,ef32a2,fee7e8"
9,version.txt,3,"3453aa,dfe85d,f1ce7f"


# Problematic files #

## Would have to be split to allow ONNX to register things in a separate file ##

* [aten/src/ATen/core/interned_strings.h](https://github.com/pytorch/pytorch/blob/master/aten/src/ATen/core/interned_strings.h): This affects `7f1705`.
* [torch/csrc/jit/python/init.cpp](https://github.com/pytorch/pytorch/blob/master/torch/csrc/jit/python/init.cpp): This affects `3f9c80,57d1df,818ce1`.
* [torch/\_C/\_\_init\_\_.pyi.in](https://github.com/pytorch/pytorch/blob/master/torch/_C/__init__.pyi.in): 	`0d5527,1de352,3f9c80,57d1df,818ce1,bfe772,c034bc`
* [tools/build_variables.bzl](https://github.com/pytorch/pytorch/blob/master/tools/build_variables.bzl): `3f9c80,7f1705`

# Probleamtic commits #

I'm not sure if these could be cleanly split into two commits

* [cab484](https://github.com/pytorch/pytorch/commit/cab4849463f962b5498b795e00886a4ed0a4dd00): this only touches things under caffe2. Would we move the caffe2 ONNX code into a new repo?
* [599f50](https://github.com/pytorch/pytorch/commit/599f5058cf0726d163e04e2d542530f63fcf439): this updated the onnx submodule.
  How would this work if we split torch.onnx into a different repo?
* [3f9c80](https://github.com/pytorch/pytorch/commit/3f9c803fe82e5bb5ccf30b82f5c129ba8c695208): 